[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Trusted-AI/AIF360/blob/main/examples/demo_json_explainers.ipynb)


In [ ]:
# Load all necessary packages
import sys
sys.path.append("../")
from collections import OrderedDict
import json
from pprint import pprint
from aif360.datasets import GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.explainers import MetricTextExplainer, MetricJSONExplainer
from IPython.display import JSON, display_json

##### Load dataset

In [ ]:
gd = GermanDataset()

##### Create metrics

In [ ]:
priv = [{'sex': 1}]
unpriv = [{'sex': 0}]
bldm = BinaryLabelDatasetMetric(gd, unprivileged_groups=unpriv, privileged_groups=priv)

##### Create explainers

In [ ]:
text_expl = MetricTextExplainer(bldm)
json_expl = MetricJSONExplainer(bldm)

##### Text explanations

In [ ]:
print(text_expl.num_positives())

Number of positive-outcome instances: 700.0


In [ ]:
print(text_expl.mean_difference())

Mean difference (mean label value on privileged instances - mean label value on unprivileged instances): -0.0748013090229


In [ ]:
print(text_expl.disparate_impact())

Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.896567328205


##### JSON Explanations

In [ ]:
def format_json(json_str):
    return json.dumps(json.loads(json_str, object_pairs_hook=OrderedDict), indent=2)

In [ ]:
print(format_json(json_expl.num_positives()))

{
  "metric": "num_positives", 
  "message": "Number of positive-outcome instances: 700.0", 
  "numPositives": 700.0, 
  "description": "Computed as the number of positive instances for the given (privileged or unprivileged) group.", 
  "ideal": "The ideal value of this metric lies in the total number of positive instances made available"
}


In [ ]:
print(format_json(json_expl.mean_difference()))

{
  "metric": "mean_difference", 
  "message": "Mean difference (mean label value on privileged instances - mean label value on unprivileged instances): -0.0748013090229", 
  "numPositivesUnprivileged": 201.0, 
  "numInstancesUnprivileged": 310.0, 
  "numPositivesPrivileged": 499.0, 
  "numInstancesPrivileged": 690.0, 
  "description": "Computed as the difference of the rate of favorable outcomes received by the unprivileged group to the privileged group.", 
  "ideal": "The ideal value of this metric is 0.0"
}


In [ ]:
print(format_json(json_expl.disparate_impact()))

{
  "metric": "disparate_impact", 
  "message": "Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.896567328205", 
  "numPositivePredictionsUnprivileged": 201.0, 
  "numUnprivileged": 310.0, 
  "numPositivePredictionsPrivileged": 499.0, 
  "numPrivileged": 690.0, 
  "description": "Computed as the ratio of likelihood of favorable outcome for the unprivileged group to that of the privileged group.", 
  "ideal": "The ideal value of this metric is 1.0"
}
